In [1]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.pandas as ps
import pandas as pd
from pyspark.sql import functions as F

conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0,com.databricks:spark-xml_2.12:0.6.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-eaedf06f-dd71-40eb-95c5-7985759d5f1c;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
	found com.databricks#spark-xml_2.12;0.6.0 in central
	found commons-io#commons-io;2.6 in central
:: resolution report :: resolve 569ms :: artifacts dl 21ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	com.databricks#spark-xml_2.12;0.6.0 from central in [default]
	com.microsoft.azure#spark-mssql-connector_2.1

In [2]:
%env S3ADRESS=s3a://daniel-3cco-poupay-poupay-bucket-estruturados
%env PASSWORD_SQL=Urubu1@@
%env USER_SQL=urubu100

env: S3ADRESS=s3a://daniel-3cco-poupay-poupay-bucket-estruturados
env: PASSWORD_SQL=Urubu1@@
env: USER_SQL=urubu100


In [3]:
import os

S3_address = os.environ['S3ADRESS']
password_sql = os.environ['PASSWORD_SQL']
user_sql = os.environ['USER_SQL']

In [17]:
df_soja = spark.read.format("com.databricks.spark.xml") \
            .option("rowTag","row")\
            .load(f"{S3_address}/dados-brutos/soja_xml")

In [18]:
df_soja.show()

+----+-------------------+-----+----+--------------------+----------+------------+---+
| ano|             estado|index|item|                nome|quantidade|      regiao| uf|
+----+-------------------+-----+----+--------------------+----------+------------+---+
|2018|          São Paulo| 5287|soja|   Redenção da Serra|      0.22|     Sudeste| SP|
|2020| Mato Grosso do Sul| 2287|soja|      Nova Andradina|      3.31|Centro-Oeste| MS|
|2007|              Piauí| 3031|soja|     Bonfim do Piauí|      2.02|    Nordeste| PI|
|2011|              Piauí| 3097|soja|            Itaueira|       0.5|    Nordeste| PI|
|2019|              Goiás|  992|soja|        Damianópolis|     11.24|Centro-Oeste| GO|
|2003|        Mato Grosso| 2406|soja|   Porto dos Gaúchos|      2.65|Centro-Oeste| MT|
|2009|              Ceará|  634|soja|             Araripe|      0.76|    Nordeste| CE|
|2003|           Maranhão| 1317|soja|       Primeira Cruz|      0.12|    Nordeste| MA|
|2002|             Paraná| 3302|soja|      

In [19]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, FloatType

_df = df_soja

_df = _df.select(
    col('ano').cast(IntegerType()).alias('ANO'),
    col('quantidade').cast(FloatType()).alias('QUANTIDADE'),
    col('estado').alias('ESTADO'),
    col('uf').alias('UF'),
    col('regiao').alias('REGIAO'),
    col('nome').alias('CIDADE')
)

_df.write \
   .mode('overwrite') \
   .csv('s3a://daniel-3cco-poupay-poupay-bucket-estruturados/dados-tratados/cidades_producao')

22/06/07 03:16:27 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/07 03:16:27 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/07 03:16:27 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [20]:
dataframe = spark.read.csv(f"{S3_address}/dados-tratados/cidades_producao")

In [21]:
dataframe.show()

+----+-----+-------------------+---+------------+--------------------+
| _c0|  _c1|                _c2|_c3|         _c4|                 _c5|
+----+-----+-------------------+---+------------+--------------------+
|2018| 0.22|          São Paulo| SP|     Sudeste|   Redenção da Serra|
|2020| 3.31| Mato Grosso do Sul| MS|Centro-Oeste|      Nova Andradina|
|2007| 2.02|              Piauí| PI|    Nordeste|     Bonfim do Piauí|
|2011|  0.5|              Piauí| PI|    Nordeste|            Itaueira|
|2019|11.24|              Goiás| GO|Centro-Oeste|        Damianópolis|
|2003| 2.65|        Mato Grosso| MT|Centro-Oeste|   Porto dos Gaúchos|
|2009| 0.76|              Ceará| CE|    Nordeste|             Araripe|
|2003| 0.12|           Maranhão| MA|    Nordeste|       Primeira Cruz|
|2002| 0.53|             Paraná| PR|         Sul|        Congonhinhas|
|2003| 0.95|       Minas Gerais| MG|     Sudeste|      Mar de Espanha|
|2018| 0.99|              Goiás| GO|Centro-Oeste|             Formoso|
|2001|

In [23]:
dataframe = dataframe.withColumnRenamed("_c0", "ano")
dataframe = dataframe.withColumnRenamed("_c1", "quantidade")
dataframe = dataframe.withColumnRenamed("_c2", "estado")
dataframe = dataframe.withColumnRenamed("_c3", "uf")
dataframe = dataframe.withColumnRenamed("_c4", "regiao")
dataframe = dataframe.withColumnRenamed("_c5", "cidade")

In [27]:
dataframe[(dataframe['regiao'] == 'Centro-Oeste') & (dataframe['ano'] == '2010')].count()

37

In [35]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, FloatType

_df = dataframe

_df = _df.select(
    col('ano').cast(IntegerType()).alias('ANO'),
    col('quantidade').cast(FloatType()).alias('QUANTIDADE'),
    col('estado').alias('ESTADO'),
    col('uf').alias('UF'),
    col('regiao').alias('REGIAO'),
    col('cidade').alias('CIDADE')
)

_df.write \
   .mode('overwrite') \
   .csv(f'{S3_address}/dados-cliente/cidades_producao')

_df.write \
 .format('com.microsoft.sqlserver.jdbc.spark') \
 .mode('overwrite') \
 .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
 .option('url', 'jdbc:sqlserver://poupay-01.database.windows.net;databaseName=poupay;') \
 .option('dbtable', 'CIDADE_PRODUTO') \
 .option('user', user_sql) \
 .option('password', password_sql) \
 .save()

22/06/07 03:23:43 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/07 03:23:43 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/07 03:23:43 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
